In [ ]:
import xarray as xr
import panel as pn
import ipyleaflet
import xarray_leaflet
from ipywidgets import Layout


pn.extension('ipywidgets')

nc_file1 = '../../data/SWE_reanalysis_32yrs.nc'
nc_file2 = '../../data/swe_reanalysis_ucrb.nc'

data1 = xr.open_dataset(nc_file1)
data2 = xr.open_dataset(nc_file2)

variable_name = 'SWE_Post'

if variable_name not in data1 or variable_name not in data2:
    raise ValueError(f"Variable '{variable_name}' not found in one of the datasets.")

def create_map():
    m = ipyleaflet.Map(center=(39.0, -120.5), zoom=6, layout=Layout(width='600px', height='600px'))
    basemap = ipyleaflet.TileLayer(url='https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', name='OpenStreetMap')
    m.add_layer(basemap)
    layer1 = data1[variable_name].isel(Year=0).leaflet.plot(m)
    layer2 = data2[variable_name].isel(Year=0).leaflet.plot(m)
    control = ipyleaflet.SplitMapControl(left_layer=layer1, right_layer=layer2)
    m.add_control(control)
    return m, control, layer1, layer2

m, control, layer1, layer2 = create_map()


def update_layers(year):
    global layer1, layer2
    if layer1 in m.layers:
        m.remove_layer(layer1)
    if layer2 in m.layers:
        m.remove_layer(layer2)
    time_index1 = data1.Year.values.tolist().index(year)
    time_index2 = data2.Year.values.tolist().index(year)
    layer1 = data1[variable_name].isel(Year=time_index1).leaflet.plot(m)
    layer2 = data2[variable_name].isel(Year=time_index2).leaflet.plot(m)
    control.left_layer = layer1
    control.right_layer = layer2
    m.add_layer(layer1)
    m.add_layer(layer2)


years = data1.Year.values
time_slider = pn.widgets.IntSlider(name='Year', start=int(years.min()), end=int(years.max()), step=1, value=int(years.min()))

def slider_callback(event):
    update_layers(event.new)

time_slider.param.watch(slider_callback, 'value')

map_pane = pn.pane.IPyWidget(object=m, height=600, width=600)

layout = pn.Row(map_pane, time_slider)

update_layers(int(years.min()))

def click_event_handler(event):
    click_data = json.loads(event.body)
    print(click_data)

def panel_app():
    pn.state.add_periodic_callback(lambda: None, period=100, count=1)
    return layout

pn.serve({'/': panel_app}, port=5102, show=True, callbacks={'/click_stream': click_event_handler})
